In [5]:
## NAME:Devyani Mardia
## NETID: dm1633

################
## Code for HW 2 problem 2
##
## INSTRUCTIONS:
## The following file implements kernel SVM with
## polynomial kernel.
##
## It also trains kernel SVM classifier on a
## dataset that contains sociodemographic information of all counties
## in the United States as well as how these counties voted in the 2016
## presidential election.
##
## The kernel SVM classifier is use to predict whether a county prefers
## Trump over Clinton based on sociodemographic features.
##
## Fill in the code in parts labeled "FILL IN".
##
################


In [10]:
## Trains kernel SVM via Projected Gradient Descent
## and with polynomial kernel
##
## INPUT: "X" n--by--p matrix,
##        "Y" n--by--1 vector of labels
##        "C" scalar, weight placed on violation reduction
##        "d" degree of polynomial kernel
## OUTPUT:

def kernelSVM(X, Y, C, d):
    p = X.shape[1]
    n = X.shape[0]
  
    variables = 1e-5 * np.ones(n)

    alpha = .1
    gamma = .9
    stepsize = 1
  
    K = (X @ X.T + 1)**d  
    ## FILL IN: compute the kernel between all pairs of training samples
    K2 = np.diag(Y) @ K @ np.diag(Y)
  
    it = 1
    while True:
        cur_obj = .5 * variables.T @ K2 @ variables - np.sum(variables)
      
        gradient = K2 @ variables - np.ones(n)
        vars_new = dualproject(variables - stepsize * gradient, Y, C)
    
        ## backtracking line search
        while (.5 * vars_new.T @ K2 @ vars_new - np.sum(vars_new) > 
               cur_obj + alpha * np.sum(gradient * (vars_new - variables))):
            stepsize = stepsize * gamma
            vars_new = dualproject(variables - stepsize * gradient, Y, C)
    
        new_obj = .5 * vars_new.T @ K2 @ vars_new - np.sum(vars_new)
    
        if it % 100 == 0:
            print("Iteration: %d  objective: %.3f  conv threshold (log10) %.3f  stepsize %.3f" % (
                  it, new_obj, 
                  np.log10(np.mean((variables - vars_new)**2 / np.mean(variables**2))),
                  stepsize
                  ))
    
        it = it + 1
        
        if np.mean((variables - vars_new)**2) / np.mean(variables**2) < 1e-6:
            break
        else:
            variables = vars_new
  
    return variables


In [11]:
## Runs Dykstra's algorithm
##
## INPUT:  u is length-n vector
##         Y is length-n vector
##         C is a positive scalar
##

def dualproject(u, Y, C):
    n = len(u)

    v = u
    p = np.zeros(n)
    q = np.zeros(n)

    T = 1000
    for it in range(T):
        w = project2(v + p, Y)
        p = v + p - w
        v = project1(w + q, C)
        q = w + q - v

        if (np.mean((v-w)**2) < 1e-20):
            break

    return v


def project1(u, C):
    return np.minimum(np.maximum(u, 0), C)

def project2(u, Y):
    return u - Y * sum(u * Y)/sum(Y**2)


In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

np.random.seed(1)

##################
##
##  Using kernel SVM to predict voting behavior in the 2016 US presidential election.
##
######

votes = pd.read_csv("votes.csv")
votes['prefer_trump'] = votes['trump'] > votes['clinton']
features = ["white", "black", "poverty", "density", "bachelor", "highschool", "age65plus", "income", "age18under", "population2014"]

X = votes[features].values
scaler = StandardScaler()
X = scaler.fit_transform(X)
Y = [int(x) for x in votes['prefer_trump'].values]
Y = np.array(Y)

ntrain = 400 
test_ix = np.random.choice(votes.index, size=votes.shape[0] - ntrain, replace=False)
learn_ix = [x for x in range(votes.shape[0]) if x not in test_ix]

Y[Y == 0] = -1

X1 = X[learn_ix, ]
Y1 = Y[learn_ix]

X2 = X[test_ix, ]
Y2 = Y[test_ix]

p = X.shape[1]

## Run SVM
C = 0.5
deg = 1 ## Modify this to 2, 3, 4, 5, and 6 for part (b)
alpha = kernelSVM(X1, Y1, C, deg)

ixs = np.where((alpha > 0) & (alpha < C))[0]

b = np.mean(Y1[ixs].reshape(-1,1) - ((X1[ixs] @ X1.T + 1)**deg)@(np.multiply(alpha,Y1).reshape(-1,1)), axis = 0) # FILL IN

Y2_svm = np.sign(((X2 @ X1.T + 1)**deg) @ np.multiply(alpha,Y1) + b)   ## FILL IN: compute the labels predicted by kernel SVM

svm_error = np.mean(np.abs(Y2_svm - Y2))/2 ## because Y2 and Y2_svm are +1/-1
num_supp_vec = len(ixs) ## FILL IN: compute the number of support vectors

# Run logistic regression
Y1[Y1 == -1] = 0
Y2[Y2 == -1] = 0

X1 = np.append(X1, np.ones((X1.shape[0], 1)), axis=1)
X2 = np.append(X2, np.ones((X2.shape[0], 1)), axis=1)

w_lr = LogisticRegression(penalty="none", solver="lbfgs", fit_intercept=False).fit(X1, Y1).coef_[0]

Y2_lr = X2 @ w_lr >= 0.0
lr_error = np.mean(np.abs(Y2_lr - Y2))

baseline_error = 1.0 - np.mean(Y2) if np.mean(Y1) > 0.5 else np.mean(Y2)

print("Baseline Error: %.3f  kernelSVM Error: %.3f  LR Error: %.3f  #Support vec: %d" % (baseline_error, svm_error, lr_error, num_supp_vec))


Iteration: 100  objective: -20.064  conv threshold (log10) -4.061  stepsize 0.001
Iteration: 200  objective: -36.969  conv threshold (log10) -4.750  stepsize 0.001
Iteration: 300  objective: -45.647  conv threshold (log10) -5.349  stepsize 0.001
Iteration: 400  objective: -48.549  conv threshold (log10) -5.925  stepsize 0.001
Baseline Error: 0.154  kernelSVM Error: 0.088  LR Error: 0.091  #Support vec: 102


In [20]:
#PART B

def runner(degree):
    print(
        f"=====================Running SVM for Degree = {degree} ============================="
    )
    np.random.seed(1)

    ##################
    ##
    ##  Using kernel SVM to predict voting behavior in the 2016 US presidential election.
    ##
    ######

    votes = pd.read_csv("votes.csv")
    votes["prefer_trump"] = votes["trump"] > votes["clinton"]
    features = [
        "white",
        "black",
        "poverty",
        "density",
        "bachelor",
        "highschool",
        "age65plus",
        "income",
        "age18under",
        "population2014",
    ]

    X = votes[features].values
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    Y = [int(x) for x in votes["prefer_trump"].values]
    Y = np.array(Y)

    ntrain = 400
    test_ix = np.random.choice(votes.index, size=votes.shape[0] - ntrain, replace=False)
    learn_ix = [x for x in range(votes.shape[0]) if x not in test_ix]

    Y[Y == 0] = -1

    X1 = X[learn_ix,]
    Y1 = Y[learn_ix]

    X2 = X[test_ix,]
    Y2 = Y[test_ix]

    p = X.shape[1]

    ## Run SVM
    C = 0.5
    deg = degree  ## Modify this to 2, 3, 4, 5, and 6 for part (b)
    alpha = kernelSVM(X1, Y1, C, deg)

    ixs = np.where((alpha > 0) & (alpha < C))[0]

    b = np.mean(
        Y1[ixs].reshape(-1, 1)
        - ((X1[ixs] @ X1.T + 1) ** deg) @ (np.multiply(alpha, Y1).reshape(-1, 1)),
        axis=0,
    )  # FILL IN

    Y2_svm = np.sign(
        ((X2 @ X1.T + 1) ** deg) @ np.multiply(alpha, Y1) + b
    )  ## FILL IN: compute the labels predicted by kernel SVM

    svm_error = np.mean(np.abs(Y2_svm - Y2)) / 2  ## because Y2 and Y2_svm are +1/-1
    num_supp_vec = len(ixs)  ## FILL IN: compute the number of support vectors

    # Run logistic regression
    Y1[Y1 == -1] = 0
    Y2[Y2 == -1] = 0

    X1 = np.append(X1, np.ones((X1.shape[0], 1)), axis=1)
    X2 = np.append(X2, np.ones((X2.shape[0], 1)), axis=1)

    w_lr = (
        LogisticRegression(penalty="none", solver="lbfgs", fit_intercept=False)
        .fit(X1, Y1)
        .coef_[0]
    )

    Y2_lr = X2 @ w_lr >= 0.0
    lr_error = np.mean(np.abs(Y2_lr - Y2))

    baseline_error = 1.0 - np.mean(Y2) if np.mean(Y1) > 0.5 else np.mean(Y2)

    print(
        "\nBaseline Error: %.3f  kernelSVM Error: %.3f  LR Error: %.3f  #Support vec: %d"
        % (baseline_error, svm_error, lr_error, num_supp_vec)
    )
    print("\n")

In [21]:
# Running for different values of degrees as asked in question:
for d in [1,2,3,4,5,6]:
    runner(d)

=====================Running SVM for Degree = 1 =============================
Iteration: 100  objective: -20.064  conv threshold (log10) -4.061  stepsize 0.001
Iteration: 200  objective: -36.969  conv threshold (log10) -4.750  stepsize 0.001
Iteration: 300  objective: -45.647  conv threshold (log10) -5.349  stepsize 0.001
Iteration: 400  objective: -48.549  conv threshold (log10) -5.925  stepsize 0.001

Baseline Error: 0.154  kernelSVM Error: 0.088  LR Error: 0.091  #Support vec: 102


=====================Running SVM for Degree = 2 =============================
Iteration: 100  objective: -1.022  conv threshold (log10) -4.066  stepsize 0.000
Iteration: 200  objective: -1.827  conv threshold (log10) -4.654  stepsize 0.000
Iteration: 300  objective: -2.583  conv threshold (log10) -5.003  stepsize 0.000
Iteration: 400  objective: -3.306  conv threshold (log10) -5.253  stepsize 0.000
Iteration: 500  objective: -4.006  conv threshold (log10) -5.446  stepsize 0.000
Iteration: 600  objective:

Run with the code with degree set to 1, then 2, 3, 4, 5, and 6. Report the predictive error as
well as the number of support vectors in each of the cases.


Baseline Error: 0.154  kernelSVM Error: 0.088  LR Error: 0.091  #Support vec: 102
Baseline Error: 0.154  kernelSVM Error: 0.080  LR Error: 0.091  #Support vec: 135
Baseline Error: 0.154  kernelSVM Error: 0.086  LR Error: 0.091  #Support vec: 235
Baseline Error: 0.154  kernelSVM Error: 0.085  LR Error: 0.091  #Support vec: 336
Baseline Error: 0.154  kernelSVM Error: 0.150  LR Error: 0.091  #Support vec: 372
Baseline Error: 0.154  kernelSVM Error: 0.143  LR Error: 0.091  #Support vec: 381

Predictive error increases as the polynomial degree increases, this can be expected as the higher degrees of freedom of the curve it is more likely to overfit the data causing the error to increase at higher degrees. It is lowest at degree=2.


How does the number of support vectors change with the degree of the polynomial kernel? Explain why.

As the degree of freedom increases the number of support vectors to fit this data also increases to give us a more smoother and tightly fitted curve to the data , hence the error increases as well indicating overfitting.